# 007 Clustering Validation: FEMA application data
* Damaged properties
* Data analysis for Figure 2
* Mapping and additional analysis have been done in Quantum GIS version 3.4 Madeira

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings("ignore")

In [2]:
# Check package versions
import types 
def imports():
    for name, val in globals().items():
        if isinstance(val, types.ModuleType):
            yield val.__name__

import pkg_resources
root_packages = [i.split('.', 1)[0] for i in list(imports())] 
for m in pkg_resources.working_set:
    if m.project_name.lower() in root_packages:
        print (m.project_name, m.version)

pandas 1.1.3
numpy 1.19.1
matplotlib 3.3.2


# 1. Load FEMA housing assisntance application data 
* For owner
* For renter

In [3]:
df_o = pd.read_csv('../../data/FEMA_Housing_Assistance_Data_Owner.csv')
df_r = pd.read_csv('../../data/FEMA_Housing_Assistance_Data_Renter.csv')

In [4]:
df_o = df_o[['Disaster', 'State', 'County', 'City', 'Zip Code', 'Valid Registrations', 
             'Average FEMA Inspected Damage', 'Total Inspected', 'Total Damage', 'No FEMA Inspected Damage', 
             'FEMA Inspected Damage between $1 and $10,000', 'FEMA Inspected Damage between $10,001 and $20,000', 
             'FEMA Inspected Damage between $20,001 and $30,000', 'FEMA Inspected Damage > $30,000', 
             'Approved for FEMA Assistance', 'Total Approved IHP Amount', 'Repair/Replace Amount', 'Rental Amount', 
             'Other Needs Amount', 'Approved between $1 and $10,000', 'Approved between $10,001 and $25,000', 
             'Approved between $25,001 and Max', 'Total Max Grants']]

df_r = df_r[['Disaster', 'State', 'County', 'City', 'Zip Code', 'Valid Registrations', 
             'Total Inspected', 'Inspected with No Damage', 'Total with Moderate Damage', 'Total with Major Damage', 
             'Total with Substantial Damage', 
             'Approved for FEMA Assistance', 'Total Approved IHP Amount', 'Repair/Replace Amount', 'Rental Amount', 
             'Other Needs Amount', 'Approved between $1 and $10,000', 'Approved between $10,001 and $25,000', 
             'Approved between $25,001 and Max', 'Total Max Grants']]

In [5]:
df_o = df_o[df_o['Disaster']==4332]
df_r = df_r[df_r['Disaster']==4332]
df_o = df_o[df_o['County']=='Harris (County)']
df_r = df_r[df_r['County']=='Harris (County)']

df_o = df_o.reset_index(drop=True)
df_r = df_r.reset_index(drop=True)

col_list = df_o.columns.tolist()[8:11]+df_o.columns.tolist()[12:]

for c in col_list:
    df_o[c] = df_o[c].astype(str)
    df_o[c] = df_o[c].str.replace('$', '')
    df_o[c] = df_o[c].str.replace(',', '')
    df_o[c] = df_o[c].astype(float)
    
for c in ['Total Approved IHP Amount', 'Repair/Replace Amount', 'Rental Amount', 'Other Needs Amount']:
    df_r[c] = df_r[c].astype(str)
    df_r[c] = df_r[c].str.replace('$', '')
    df_r[c] = df_r[c].str.replace(',', '')
    df_r[c] = df_r[c].astype(float)

In [6]:
df_o.head(2)

,Disaster,State,County,City,Zip Code,Valid Registrations,Average FEMA Inspected Damage,Total Inspected,Total Damage,No FEMA Inspected Damage,...,"FEMA Inspected Damage > $30,000",Approved for FEMA Assistance,Total Approved IHP Amount,Repair/Replace Amount,Rental Amount,Other Needs Amount,"Approved between $1 and $10,000","Approved between $10,001 and $25,000","Approved between $25,001 and Max",Total Max Grants
0,4332.0,TX,Harris (County),AMARILLO,79103.0,1,0.00,1.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,4332.0,TX,Harris (County),ARCOLA,77583.0,1,0.00,1.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [7]:
df_r.head(2)

,Disaster,State,County,City,Zip Code,Valid Registrations,Total Inspected,Inspected with No Damage,Total with Moderate Damage,Total with Major Damage,Total with Substantial Damage,Approved for FEMA Assistance,Total Approved IHP Amount,Repair/Replace Amount,Rental Amount,Other Needs Amount,"Approved between $1 and $10,000","Approved between $10,001 and $25,000","Approved between $25,001 and Max",Total Max Grants
0,4332.0,TX,Harris (County),ABILENE,79606.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,4332.0,TX,Harris (County),ALICE,78332.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [8]:
df_o['Valid Registrations'] = df_o['Valid Registrations'].astype(float)
df_o = df_o.groupby('Zip Code').sum().reset_index(inplace=False)
df_r = df_r.groupby('Zip Code').sum().reset_index(inplace=False)

df_o = df_o[['Zip Code', 'Valid Registrations', 'Total Inspected', 'Total Damage', 'No FEMA Inspected Damage', 
            'Approved for FEMA Assistance','Total Approved IHP Amount']]

df_r = df_r[['Zip Code', 'Valid Registrations', 'Total Inspected', 'Inspected with No Damage', 'Approved for FEMA Assistance',
            'Total Approved IHP Amount']]

# print len(df_o)
# print len(df_r)

In [9]:
df_r.head()

,Zip Code,Valid Registrations,Total Inspected,Inspected with No Damage,Approved for FEMA Assistance,Total Approved IHP Amount
0,0.0,1.0,0.0,0.0,0.0,0.0
1,37001.0,1.0,0.0,0.0,0.0,0.0
2,46615.0,1.0,0.0,0.0,0.0,0.0
3,70042.0,1.0,0.0,0.0,0.0,0.0
4,70044.0,1.0,1.0,1.0,0.0,0.0


In [10]:
df_o.head()

,Zip Code,Valid Registrations,Total Inspected,Total Damage,No FEMA Inspected Damage,Approved for FEMA Assistance,Total Approved IHP Amount
0,70076.0,1.0,1.0,226.0,0.0,0.0,0.0
1,70811.0,1.0,1.0,0.0,1.0,0.0,0.0
2,75067.0,1.0,1.0,0.0,1.0,0.0,0.0
3,75074.0,1.0,1.0,0.0,1.0,0.0,0.0
4,75224.0,1.0,1.0,0.0,1.0,1.0,500.0


# 2. Validation of damage for each clusters
* Calculate damaged properties per household at zipcode level

In [11]:
# Num of households (zipcode)
hh_zip = pd.read_csv('../../data/household_zipcode_Texas/ACS_17_5YR_S1101_with_ann.csv', skiprows=[0]) 
# Num of households (census tract)
hh_ct = pd.read_csv('../../data/household_census_tract_Harris_county/ACS_17_5YR_S1101_with_ann.csv', skiprows=[0])
# Zipcode/census tract IDs
ct_zip = pd.read_csv('../../data/ct_zip.csv')

In [12]:
df_r['insp_damage_r'] = df_r['Total Inspected'] - df_r['Inspected with No Damage']
df_o['insp_damage_o'] = df_o['Total Inspected'] - df_o['No FEMA Inspected Damage']

In [13]:
df_damage = pd.merge(df_o, df_r, how='outer', on='Zip Code')
df_damage = df_damage[['Zip Code', 'insp_damage_r', 'insp_damage_o']]
df_damage = df_damage.fillna(0)
df_damage['insp_damage'] = df_damage['insp_damage_o'] + df_damage['insp_damage_r']

df_damage.columns = ['zipcode', 'insp_damage_r', 'insp_damage_o', 'insp_damage']
df_damage.head()

,zipcode,insp_damage_r,insp_damage_o,insp_damage
0,70076.0,0.0,1.0,1.0
1,70811.0,0.0,0.0,0.0
2,75067.0,0.0,0.0,0.0
3,75074.0,0.0,0.0,0.0
4,75224.0,0.0,0.0,0.0


In [14]:
hh_zip = hh_zip[['Id2', 'Total; Estimate; Total households']]
hh_zip.columns = ['zipcode', 'hh']
df_damage_zip = pd.merge(df_damage, hh_zip, how='outer', on='zipcode')
df_damage_zip = df_damage_zip.dropna()
df_damage_zip['damage_per_hh'] = df_damage_zip['insp_damage']/df_damage_zip['hh']
df_damage_zip.head(2)

,zipcode,insp_damage_r,insp_damage_o,insp_damage,hh,damage_per_hh
2,75067.0,0.0,0.0,0.0,24913.0,0.0
3,75074.0,0.0,0.0,0.0,16430.0,0.0


In [15]:
df_damage_zip = df_damage_zip[['zipcode', 'damage_per_hh']]

In [16]:
df_damage_zip.head()

,zipcode,damage_per_hh
2,75067.0,0.0
3,75074.0,0.0
4,75224.0,0.0
5,75234.0,0.0
6,75402.0,0.0


In [17]:
# df_damage_zip.to_csv('../../outputs/damage_per_hh_zip.csv')

* Calculate number of damaged properties at census tract level

In [18]:
ct_zip['TRACT'] = ct_zip['TRACT'].astype(str).str.zfill(6)
ct_zip['GEOID_Data'] = '14000US48201' + ct_zip['TRACT']
ct_zip.head(2)

,COUNTY,TRACT,zipcode,GEOID_Data
0,201,100000,77002,14000US48201100000
1,201,210100,77002,14000US48201210100


In [19]:
df_damage_ct = pd.merge(ct_zip, df_damage_zip, how='left', on='zipcode')
# print len(df_damage_ct)
# print df_damage_ct['GEOID_Data'].nunique()
df_damage_ct.head()

,COUNTY,TRACT,zipcode,GEOID_Data,damage_per_hh
0,201,100000,77002,14000US48201100000,0.012140
1,201,210100,77002,14000US48201210100,0.012140
2,201,210400,77009,14000US48201210400,0.046346
3,201,210500,77009,14000US48201210500,0.046346
4,201,210600,77022,14000US48201210600,0.103706


In [20]:
hh_ct = hh_ct[['Id2', 'Total; Estimate; Total households']]
hh_ct['GEOID_Data'] = '14000US' + hh_ct['Id2'].astype(str)
hh_ct['hh'] = hh_ct['Total; Estimate; Total households']
hh_ct = hh_ct[['GEOID_Data', 'hh']]
hh_ct.head(2)

,GEOID_Data,hh
0,14000US48201100000,2239
1,14000US48201210100,76


In [21]:
df_damage_ct = pd.merge(df_damage_ct, hh_ct, how='left', on='GEOID_Data')
df_damage_ct.head()

,COUNTY,TRACT,zipcode,GEOID_Data,damage_per_hh,hh
0,201,100000,77002,14000US48201100000,0.012140,2239
1,201,210100,77002,14000US48201210100,0.012140,76
2,201,210400,77009,14000US48201210400,0.046346,1609
3,201,210500,77009,14000US48201210500,0.046346,1849
4,201,210600,77022,14000US48201210600,0.103706,2038


In [22]:
df_damage_ct['damage_ct'] = df_damage_ct['damage_per_hh'] * df_damage_ct['hh']
df_damage_ct.head()

,COUNTY,TRACT,zipcode,GEOID_Data,damage_per_hh,hh,damage_ct
0,201,100000,77002,14000US48201100000,0.012140,2239,27.180577
1,201,210100,77002,14000US48201210100,0.012140,76,0.922610
2,201,210400,77009,14000US48201210400,0.046346,1609,74.570612
3,201,210500,77009,14000US48201210500,0.046346,1849,85.693637
4,201,210600,77022,14000US48201210600,0.103706,2038,211.353056


In [23]:
#df_damage_ct.to_csv('../../outputs/damage_ct.csv', index=False)